In [1]:
# importing required libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
# Load the dataset
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
df = pd.read_csv('/content/drive/MyDrive/Portfolio/1. Capstone Projects/Galaxy Classification(DL Project)/GalaxyZoo1_DR_table2.csv')
df.head()

,OBJID,RA,DEC,NVOTE,P_EL,P_CW,P_ACW,P_EDGE,P_DK,P_MG,P_CS,P_EL_DEBIASED,P_CS_DEBIASED,SPIRAL,ELLIPTICAL,UNCERTAIN
0,587727178986356823,00:00:00.41,-10:22:25.7,59,0.610,0.034,0.000,0.153,0.153,0.051,0.186,0.610,0.186,0,0,1
1,587727227300741210,00:00:00.74,-09:13:20.2,18,0.611,0.000,0.167,0.222,0.000,0.000,0.389,0.203,0.797,1,0,0
2,587727225153257596,00:00:01.03,-10:56:48.0,68,0.735,0.029,0.000,0.147,0.074,0.015,0.176,0.432,0.428,0,0,1
3,587730774962536596,00:00:01.38,+15:30:35.3,52,0.885,0.019,0.000,0.058,0.019,0.019,0.077,0.885,0.077,0,1,0
4,587731186203885750,00:00:01.55,-00:05:33.3,59,0.712,0.000,0.000,0.220,0.068,0.000,0.220,0.640,0.290,0,0,1


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 667944 entries, 0 to 667943
Data columns (total 16 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   OBJID          667944 non-null  int64  
 1   RA             667944 non-null  object 
 2   DEC            667944 non-null  object 
 3   NVOTE          667944 non-null  int64  
 4   P_EL           667944 non-null  float64
 5   P_CW           667944 non-null  float64
 6   P_ACW          667944 non-null  float64
 7   P_EDGE         667944 non-null  float64
 8   P_DK           667944 non-null  float64
 9   P_MG           667944 non-null  float64
 10  P_CS           667944 non-null  float64
 11  P_EL_DEBIASED  667944 non-null  float64
 12  P_CS_DEBIASED  667944 non-null  float64
 13  SPIRAL         667944 non-null  int64  
 14  ELLIPTICAL     667944 non-null  int64  
 15  UNCERTAIN      667944 non-null  int64  
dtypes: float64(9), int64(5), object(2)
memory usage: 81.5+ MB


In [5]:
df.describe()

,OBJID,NVOTE,P_EL,P_CW,P_ACW,P_EDGE,P_DK,P_MG,P_CS,P_EL_DEBIASED,P_CS_DEBIASED,SPIRAL,ELLIPTICAL,UNCERTAIN
count,6.679440e+05,667944.000000,667944.000000,667944.000000,667944.000000,667944.000000,667944.000000,667944.000000,667944.000000,667944.000000,667944.000000,667944.000000,667944.000000,667944.000000
mean,5.878143e+17,38.760986,0.535947,0.074007,0.080736,0.205066,0.072924,0.031244,0.359850,0.393508,0.483439,0.284792,0.093107,0.622102
std,1.796023e+14,13.827837,0.284315,0.174440,0.179102,0.216642,0.085788,0.089369,0.298643,0.301595,0.321747,0.451316,0.290582,0.484862
min,5.877230e+17,4.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,5.877320e+17,28.000000,0.306000,0.000000,0.000000,0.054000,0.018000,0.000000,0.102000,0.106000,0.180000,0.000000,0.000000,0.000000
50%,5.877386e+17,34.000000,0.581000,0.016000,0.021000,0.130000,0.053000,0.000000,0.267000,0.347000,0.462000,0.000000,0.000000,1.000000
75%,5.877428e+17,51.000000,0.783000,0.051000,0.062000,0.275000,0.103000,0.020000,0.586000,0.664000,0.789000,1.000000,0.000000,1.000000
max,5.888489e+17,94.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [6]:
# Preprocess the data
X = df.drop(['OBJID', 'RA', 'DEC', 'SPIRAL', 'ELLIPTICAL', 'UNCERTAIN'], axis=1)
y = df[['SPIRAL', 'ELLIPTICAL', 'UNCERTAIN']]

In [7]:
y

,SPIRAL,ELLIPTICAL,UNCERTAIN
0,0,0,1
1,1,0,0
2,0,0,1
3,0,1,0
4,0,0,1
...,...,...,...
667939,1,0,0
667940,0,0,1
667941,0,0,1
667942,0,0,1


In [8]:
# Convert labels to numerical values
label_encoder = LabelEncoder()
y = y.apply(label_encoder.fit_transform)
y

,SPIRAL,ELLIPTICAL,UNCERTAIN
0,0,0,1
1,1,0,0
2,0,0,1
3,0,1,0
4,0,0,1
...,...,...,...
667939,1,0,0
667940,0,0,1
667941,0,0,1
667942,0,0,1


In [9]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((534355, 10), (133589, 10), (534355, 3), (133589, 3))

In [ ]:
# Define the model architecture
model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    layers.Dense(64, activation='relu'),
    layers.Dense(3, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=1)


Epoch 1/10
16699/16699 [==============================] - 66s 4ms/step - loss: 0.3001 - accuracy: 0.8684
Epoch 2/10
16699/16699 [==============================] - 60s 4ms/step - loss: 0.2475 - accuracy: 0.8899
Epoch 3/10
16699/16699 [==============================] - 62s 4ms/step - loss: 0.2342 - accuracy: 0.8947
Epoch 4/10
16699/16699 [==============================] - 62s 4ms/step - loss: 0.2271 - accuracy: 0.8971
Epoch 5/10
16699/16699 [==============================] - 59s 4ms/step - loss: 0.2229 - accuracy: 0.8981
Epoch 6/10
16699/16699 [==============================] - 61s 4ms/step - loss: 0.2209 - accuracy: 0.8990
Epoch 7/10
16699/16699 [==============================] - 61s 4ms/step - loss: 0.2196 - accuracy: 0.8994
Epoch 8/10
16699/16699 [==============================] - 60s 4ms/step - loss: 0.2185 - accuracy: 0.8996
Epoch 9/10
16699/16699 [==============================] - 60s 4ms/step - loss: 0.2174 - accuracy: 0.9002
Epoch 10/10
16699/16699 [==============================

In [ ]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)


4175/4175 [==============================] - 18s 3ms/step - loss: 0.2145 - accuracy: 0.9004
Test Loss: 0.21454665064811707
Test Accuracy: 0.9004409313201904


### Testing for new data

In [ ]:
n = int(input('Enter the row number'))
y_test.iloc[n:n+1,:]

,SPIRAL,ELLIPTICAL,UNCERTAIN
194710,0,1,0


In [ ]:
arr = X_test.iloc[n:n+1,:].values.astype(list)
x_list = arr[0].tolist()
x_list

[45.0, 0.844, 0.0, 0.0, 0.022, 0.133, 0.0, 0.022, 0.844, 0.022]

In [ ]:
predict = model.predict([x_list])
val_list = []
for n in range(3):
    val_list.append(predict[0][n])

target = np.max(val_list)
tolerance = 1e-6

index = None
for i, val in enumerate(val_list):
    if abs(val - target) < tolerance:
        index = i
        break
if index ==0:
    print('The galaxy is spiral in shape')
elif index==1:
    print('The galaxy is elliptical in shape')
else:
    print("The shape of galaxy can't be identified")

1/1 [==============================] - 0s 100ms/step
The galaxy is elliptical in shape
